In [1]:
import torch
import numpy as np
import dgl

In [2]:
# 0. Set Device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
dataset_args = dict()

# 1. Prepare Data
dataset_args['file']                    = r'C:\Users\Noahc\Documents\USYD\PHD\8 - Github\GNOT\data\steady_cavity_case_b200_maxU100ms_simple_normalized.npy'
dataset_args['percent split (decimal)'] = 0.7
dataset_args['randomizer seed']         = 42
dataset_args['use-normalizer']          = 'unit'
dataset_args['normalize_x']             = 'unit'
dataset_args['cell to pointwise']       = True
dataset_args['add boundaries']          = True
dataset_args['subsampler'] = 4

In [4]:
from data_storage.cavity_2d_data_handling import Cavity_2D_dataset_handling_v2
from data_utils import MIODataLoader, WeightedLpLoss, LpLoss
dgl_dataset = Cavity_2D_dataset_handling_v2(dataset_args['file'], name='cavity', train=True, sub_x = dataset_args['subsampler'],
                                    normalize_y=dataset_args['use-normalizer'], normalize_x = dataset_args['normalize_x'],
                                    data_split = dataset_args['percent split (decimal)'], seed = dataset_args['randomizer seed'],
                                    vertex = dataset_args['cell to pointwise'], boundaries = dataset_args['add boundaries']
                                    )
dgl_dataloader = MIODataLoader(dgl_dataset, batch_size=4, shuffle=False, drop_last=False)

Dataset Shape: (200, 65, 65, 3), subsampled by 4
Dataset Split up using torch generator seed: 42
              This can be replicated e.g.
                generator_object = torch.Generator().manual_seed(42)
 
torch.Size([65, 65])
Queries torch.Size([4225, 2]) Coordinates torch.Size([4225, 1])
Target features are normalized using unit transformer

c:\Users\Noahc\AppData\Local\Programs\Python\Python38\lib\site-packages\dgl\heterograph.py:92: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning(



tensor([[ 0.0093,  0.0004, -0.0328]]) tensor([[0.2326, 0.1816, 0.1040]])
Target features are normalized using unit transformer
Input features are normalized using unit transformer


In [5]:
dgl_dataset.config

{'input_dim': 2, 'theta_dim': 1, 'output_dim': 3, 'branch_sizes': [1]}

In [6]:
for dgl_batch in dgl_dataloader:
    g1, u_p1, g_u1 = dgl_batch
    break

Now what does the model see when the input is unpacked and feed into `block`

In [7]:
print('step 1: ', g1)
gs = dgl.unbatch(g1)
print('step 2: ', gs)
x = [_g.ndata['x'] for _g in gs]
print('step 3: ', len(x), 'example', x[0].shape)
x = torch.nn.utils.rnn.pad_sequence(x)
print('step 4: ', x.shape)
x = x.permute(1, 0, 2)  # B, T1, F
print('step 5: ', x.shape)
#x = torch.nn.utils.rnn.pad_sequence([_g.ndata['x'] for _g in gs]).permute(1, 0, 2)  # B, T1, F
x = torch.cat([x, u_p1.unsqueeze(1).repeat([1, x.shape[1], 1])], dim=-1)
print('step 6: ', x.shape)

branch_sizes = dgl_dataset.config['branch_sizes']
if branch_sizes:
    n_inputs = len(branch_sizes)
else: 
    n_inputs = 0

step 1:  Graph(num_nodes=16900, num_edges=0,
      ndata_schemes={'x': Scheme(shape=(2,), dtype=torch.float32), 'y': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={})
step 2:  [Graph(num_nodes=4225, num_edges=0,
      ndata_schemes={'x': Scheme(shape=(2,), dtype=torch.float32), 'y': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={}), Graph(num_nodes=4225, num_edges=0,
      ndata_schemes={'x': Scheme(shape=(2,), dtype=torch.float32), 'y': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={}), Graph(num_nodes=4225, num_edges=0,
      ndata_schemes={'x': Scheme(shape=(2,), dtype=torch.float32), 'y': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={}), Graph(num_nodes=4225, num_edges=0,
      ndata_schemes={'x': Scheme(shape=(2,), dtype=torch.float32), 'y': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={})]
step 3:  4 example torch.Size([4225, 2])
step 4:  torch.Size([4225, 4, 2])
step 5:  torch.Size([4, 4225, 2])
step 6:

So even though there are a few transformations. It looks like the batch is taken in quite normally with shape `[batch,nodes,channels]`.

Perhaps it is the loss function that is causing a difference in training.

## Model

In [8]:
from models.cgpt import CGPTNO
# 2. Construct Model
model_args = dict()
model_args['trunk_size']        = dgl_dataset.config['input_dim']
model_args['theta_size']        = dgl_dataset.config['theta_dim']
model_args['branch_sizes']      = dgl_dataset.config['branch_sizes']

model_args['output_size']         = 3
model_args['n_layers']            = 3
model_args['n_hidden']            = 128  
model_args['n_head']              = 1
model_args['attn_type']           = 'linear'
model_args['ffn_dropout']         = 0.0
model_args['attn_dropout']        = 0.0
model_args['mlp_layers']          = 2
model_args['act']                 = 'gelu'
model_args['hfourier_dim']        = 0

model_dgl = None
model_dgl = CGPTNO(
            trunk_size          = model_args['trunk_size'] + model_args['theta_size'],
            branch_sizes        = model_args['branch_sizes'],     # No input function means no branches
            output_size         = model_args['output_size'],
            n_layers            = model_args['n_layers'],
            n_hidden            = model_args['n_hidden'],
            n_head              = model_args['n_head'],
            attn_type           = model_args['attn_type'],
            ffn_dropout         = model_args['ffn_dropout'],
            attn_dropout        = model_args['attn_dropout'],
            mlp_layers          = model_args['mlp_layers'],
            act                 = model_args['act'],
            horiz_fourier_dim   = model_args['hfourier_dim']
            ).to(device)

In [9]:
out = model_dgl(g1, u_p1, g_u1)
out.shape

torch.Size([16900, 3])

In [10]:
loss_func = WeightedLpLoss(p=2,component='all', normalizer=None)

y_pred, y = out.squeeze(), g1.ndata['y'].squeeze()
print('step 7.1: ', 'before:', out.shape, 'after:', y_pred.shape)
print('step 7.2: ', 'before:', g1.ndata['y'].shape, 'after:', y.shape)

# 5.2. Calculate Loss for Backwards Pass
loss, reg,  _ = loss_func(g1, y_pred, y)
print('step 7.3: ', 'loss:', loss, 'reg:', reg)
loss_total = loss + reg
print('step 7.4: ', 'total loss:', loss_total)

step 7.1:  before: torch.Size([16900, 3]) after: torch.Size([16900, 3])
step 7.2:  before: torch.Size([16900, 3]) after: torch.Size([16900, 3])
step 7.3:  loss: tensor(0.8750, grad_fn=<MeanBackward0>) reg: tensor(0.)
step 7.4:  total loss: tensor(0.8750, grad_fn=<AddBackward0>)


SO the output is concatenated along the batches. Let's see if the loss looks different if we change the layout

In [11]:
# 5.2. Calculate Loss for Backwards Pass
loss, reg,  _ = loss_func(g1, y_pred.reshape(4,65,65,3), y.reshape(4,65,65,3))
print('step 8.1: ', 'loss:', loss, 'reg:', reg)
loss_total = loss + reg
print('step 8.2: ', 'total loss:', loss_total)

DGLError: Expect number of features to match number of nodes (len(u)). Got 4 and 16900 instead.

Nope does not work due to their weird function. Lets see if we can replicate the function

In [12]:
mse_loss = torch.nn.functional.mse_loss(y_pred, y, reduction = 'mean')
print('step 9: ', 'total loss:', mse_loss)

step 9:  total loss: tensor(0.8342, grad_fn=<MseLossBackward0>)


This looks very similar. In the paper they use relative L2 norm as a metric. Let's see if that works too

In [13]:
lploss_f = LpLoss()
lploss = lploss_f.rel(y_pred, y)
print('step 10: ', 'total loss:', lploss)

step 10:  total loss: tensor(1.3175, grad_fn=<MeanBackward0>)


These are all still in the relative ball-park of losses. However, we are still looking at a series of points (all batches concatenated). We want the loss to be independent of batch.

In [14]:
mse_loss = torch.nn.functional.mse_loss(y_pred.reshape(4,65,65,3), y.reshape(4,65,65,3), reduction = 'mean')
print('step 11: ', 'total loss:', mse_loss)
lploss = lploss_f.rel(y_pred.reshape(4,65,65,3), y.reshape(4,65,65,3))
print('step 12: ', 'total loss:', lploss)

step 11:  total loss: tensor(0.8342, grad_fn=<MseLossBackward0>)
step 12:  total loss: tensor(1.0384, grad_fn=<MeanBackward0>)


So MSE is independent of matrix structure but rel LP loss isn't.

Lets change the batchsize and see if that has an effect.

In [15]:
mse_loss = torch.nn.functional.mse_loss(y_pred.reshape(4,65,65,3)[:2,...], y.reshape(4,65,65,3)[:2,...], reduction = 'mean')
print('step 13: ', 'total loss:', mse_loss)
lploss = lploss_f.rel(y_pred.reshape(4,65,65,3)[:2,...], y.reshape(4,65,65,3)[:2,...])
print('step 14: ', 'total loss:', lploss)

step 13:  total loss: tensor(0.8278, grad_fn=<MseLossBackward0>)
step 14:  total loss: tensor(1.0400, grad_fn=<MeanBackward0>)


Not really, that is a good sign! Question is though, why did the training not converge or break when using our dgl-free dataloader. Could it have been due to the normalization or the use of Reynolds number instead of Lid Velocities?

Finally, lets run it through the model and see if a reduced batchsize has an effect there.

In [16]:
dgl_dataloader2 = MIODataLoader(dgl_dataset, batch_size=2, shuffle=False, drop_last=False)

for dgl_batch in dgl_dataloader2:
    g1, u_p1, g_u1 = dgl_batch
    break

out = model_dgl(g1, u_p1, g_u1)

y_pred, y = out.squeeze(), g1.ndata['y'].squeeze()
print('step 15.1: ', 'before:', out.shape, 'after:', y_pred.shape)
print('step 15.2: ', 'before:', g1.ndata['y'].shape, 'after:', y.shape)

# 5.2. Calculate Loss for Backwards Pass
loss, reg,  _ = loss_func(g1, y_pred, y)
print('step 15.3: ', 'loss:', loss, 'reg:', reg)
loss_total = loss + reg
print('step 15.4: ', 'total loss:', loss_total)

step 15.1:  before: torch.Size([8450, 3]) after: torch.Size([8450, 3])
step 15.2:  before: torch.Size([8450, 3]) after: torch.Size([8450, 3])
step 15.3:  loss: tensor(0.8743, grad_fn=<MeanBackward0>) reg: tensor(0.)
step 15.4:  total loss: tensor(0.8743, grad_fn=<AddBackward0>)


No difference. The model+loss combination is not impacted by batchsize. While the methods produce slightly different losses, they are all within the same range too, so no large red flags as of yet.

## Next Step: Recreate Dataloader
Recreate DGL Dataloader exactly but DGL

In [17]:
from data_utils import MultipleTensors
from utils import UnitTransformer

class Cavity_2D_dataset_handling_v3():
    def __init__(self, data_path, L=1.0, name=' ', sub_x = 1, train=True, normalize_y=False, y_normalizer=None, normalize_x = False, x_normalizer = None, up_normalizer =None, 
                 data_split = 0.7, seed = 42, vertex = False, boundaries = False):

        # Normalizer settings:
        self.normalize_y = normalize_y
        self.normalize_x = normalize_x
        self.y_normalizer = y_normalizer
        self.x_normalizer = x_normalizer
        self.up_normalizer = up_normalizer
        self.sub_x = sub_x # <- not implemented yet
        self.seed = seed
        self.data_split = data_split
        self.vertex = vertex
        self.boundaries = boundaries

        # Load in Dataset and retrieve shape
        self.data_out   = np.load(data_path)
        if self.sub_x > 1: self.subsampler()
        if self.vertex: self.cell_to_vertex_converter()
        if self.boundaries: self.add_boundaries()

        print(f'Dataset Shape: {self.data_out.shape}, subsampled by {self.sub_x}')
        # NOTE this can also be in the form of reynolds number 
        self.data_lid_v = np.round(np.arange(0.5,100.5,0.5),1) # * 0.1/0.01 <- apply for Reynolds Number
        self.n_batches  = self.data_out.shape[0]
        self.nx         = int(self.data_out.shape[1])
        self.num_nodes  = self.nx**2

        self.L = L
        self.train = train

        #super(Cavity_2D_dataset_handling_v3, self)#.__init__(name)   # invoke super method after read data
        self.process()

    def process(self):
        
        # SECTION 0: Split into train or test (Same as for FNO training)
        train_size = int(self.data_split * self.n_batches)
        test_size = self.n_batches - train_size

        seed_generator = torch.Generator().manual_seed(self.seed)

        train_dataset,  test_dataset    = torch.utils.data.random_split(torch.from_numpy(self.data_out),    [train_size, test_size], generator=seed_generator)
        train_lid_v,    test_lid_v      = torch.utils.data.random_split(torch.from_numpy(self.data_lid_v),  [train_size, test_size], generator=seed_generator)
        
        print(f'''Dataset Split up using torch generator seed: {seed_generator.initial_seed()}
              This can be replicated e.g.
                generator_object = torch.Generator().manual_seed({seed_generator.initial_seed()})\n ''')
        
        # The torch.utils.data.random_split() only gives objects with the whole datset or a integers, so we need to override these variables with the indexed datset split
        train_dataset,  test_dataset    = train_dataset.dataset[train_dataset.indices,...], test_dataset.dataset[test_dataset.indices,...]
        train_lid_v,    test_lid_v      = train_lid_v.dataset[train_lid_v.indices], test_lid_v.dataset[test_lid_v.indices]

        if self.train:
            self.data_out   = train_dataset
            self.data_lid_v = train_lid_v
            self.n_batches  = train_size
        else:
            self.data_out   = test_dataset
            self.data_lid_v = test_lid_v
            self.n_batches  = test_size
        
        # SECTION 1: Transformer Queries
        # Assume Isotropic Grid adjusting coordinates for cell centered or vertex points accordingly.
        # Also includes boundaries if stated (note boundaries + cell-centered will cause boundary coordinates to be 0-dx, 1+dx overflow)
        # this is to maintain isotropic property
        divisor = self.nx - 2*int(self.boundaries) + 1*int(self.vertex)
        dx = self.L/divisor
        offset = dx/2 - dx*int(self.boundaries) + dx/2*int(self.vertex)
        x = torch.arange(self.nx)/divisor + offset
        y = x

        # take note of the indexing. Best for this to match the output
        [X, Y] = torch.meshgrid(x, y, indexing = 'ij')
        print(X.shape)
        X = X.reshape(self.num_nodes,1)
        Y = Y.reshape(self.num_nodes,1)

        # we need to linearize these matrices.
        self.X_for_queries = torch.concat([Y,X],dim=-1)
        print('Queries', self.X_for_queries.shape, 'Coordinates', X.shape)
        
        # Final Data:
        self.queries = self.X_for_queries
        self.theta = torch.zeros([self.n_batches])
        self.input_f = self.data_lid_v
        self.output_truth = self.data_out.reshape(self.n_batches, self.num_nodes,3)

        
        print('Queries:',self.queries.shape)
        print('Theta: (Zeros)',self.theta.shape)
        print('Input Queries:',self.input_f.shape)
        print('Ground Truth:',self.output_truth.shape)

        # SECTION 3: Transform to be MIOdataset Loader Compatible
        self.dgl_free_processeor()
        self.__update_dataset_config()

    def subsampler(self):
        self.data_out = torch.nn.functional.avg_pool2d(torch.tensor(self.data_out).permute(0,3,1,2), self.sub_x).permute(0,2,3,1).numpy()

    def cell_to_vertex_converter(self):
        self.data_out = torch.nn.functional.avg_pool2d(torch.tensor(self.data_out).permute(0,3,1,2),2,stride=1).permute(0,2,3,1).numpy()
    
    def add_boundaries(self):
        self.data_out = torch.nn.functional.pad(torch.tensor(self.data_out).permute(0,3,1,2),(1, 1, 1, 1)).permute(0,2,3,1).numpy()

        # Lid Velocity
        self.data_out[:,-1 ,:,0] = 1

        # Pressure
        self.data_out[:,  0 ,1:-1, 2] = self.data_out[:,  1 ,1:-1, 2]  # Bottom Wall
        self.data_out[:, -1 ,1:-1, 2] = self.data_out[:, -2 ,1:-1, 2]  # Lid (y-vel)
        self.data_out[:,1:-1,  0 , 2] = self.data_out[:,1:-1,  1 , 2]  # Left Wall
        self.data_out[:,1:-1, -1 , 2] = self.data_out[:,1:-1, -2 , 2]  # Right Wall

    # Now we get to our own processor
    def dgl_free_processeor(self):

        if self.normalize_y:
            print('normalizing y (output truth)')
            self.__normalize_y()
        if self.normalize_x:
            print('normalizing x (input coordinates)')
            self.__normalize_x()

        self.__update_dataset_config()

    # This appears to normalize the entire dataset vertically stacked
    def __normalize_y(self):
        if self.y_normalizer is None:
            #y_feats_all = torch.cat([g.ndata['y'] for g in self.graphs],dim=0)
            y_feats_all = self.output_truth.reshape(self.n_batches * self.num_nodes,3)
            if self.normalize_y == 'unit':
                self.y_normalizer = UnitTransformer(y_feats_all)
                print('Target features are normalized using unit transformer')
                print(self.y_normalizer.mean, self.y_normalizer.std)
            else: 
                raise NotImplementedError
        
        self.output_truth = self.y_normalizer.transform(self.output_truth, inverse=False)
        #for g in self.graphs:
        #    g.ndata['y'] = self.y_normalizer.transform(g.ndata["y"], inverse=False)  # a torch quantile transformer
        print('Target features are normalized using unit transformer')

    def __normalize_x(self):
        if self.x_normalizer is None:
            # X features are the same for all cases (same grid coords)
            #x_feats_all = torch.cat([g.ndata["x"] for g in self.graphs],dim=0)
            x_feats_all = self.queries
            if self.normalize_x == 'unit':
                self.x_normalizer = UnitTransformer(x_feats_all)
                self.up_normalizer = UnitTransformer(self.theta)
            else: 
                raise NotImplementedError

        #for g in self.graphs:
        #    g.ndata['x'] = self.x_normalizer.transform(g.ndata['x'], inverse=False)
        self.queries = self.x_normalizer.transform(self.queries, inverse=False)
        self.u_p_list = self.up_normalizer.transform(self.theta, inverse=False)
        print('Input features are normalized using unit transformer')

    def __update_dataset_config(self):
        self.config = {
            'input_dim': self.queries.shape[-1],
            'theta_dim': 1,
            'output_dim': self.output_truth.shape[-1],
            'branch_sizes': [1]
        }

In [18]:
dgl_free_dataset = Cavity_2D_dataset_handling_v3(dataset_args['file'], name='cavity', train=True, sub_x = dataset_args['subsampler'],
                                    normalize_y=dataset_args['use-normalizer'], normalize_x = dataset_args['normalize_x'],
                                    data_split = dataset_args['percent split (decimal)'], seed = dataset_args['randomizer seed'],
                                    vertex = dataset_args['cell to pointwise'], boundaries = dataset_args['add boundaries']
                                    )

Dataset Shape: (200, 65, 65, 3), subsampled by 4
Dataset Split up using torch generator seed: 42
              This can be replicated e.g.
                generator_object = torch.Generator().manual_seed(42)
 
torch.Size([65, 65])
Queries torch.Size([4225, 2]) Coordinates torch.Size([4225, 1])
Queries: torch.Size([4225, 2])
Theta: (Zeros) torch.Size([140])
Input Queries: torch.Size([140])
Ground Truth: torch.Size([140, 4225, 3])
normalizing y (output truth)
Target features are normalized using unit transformer
tensor([[ 0.0093,  0.0004, -0.0328]], dtype=torch.float64) tensor([[0.2326, 0.1816, 0.1040]], dtype=torch.float64)
Target features are normalized using unit transformer
normalizing x (input coordinates)
Input features are normalized using unit transformer


Let's compare unit normalizer statistics:

In [19]:
print('dgl free (x):',dgl_free_dataset.x_normalizer.mean, dgl_free_dataset.x_normalizer.std)
print('dgl      (x):',dgl_dataset.x_normalizer.mean, dgl_dataset.x_normalizer.std)

dgl free (x): tensor([[0.5000, 0.5000]]) tensor([[0.2932, 0.2932]])
dgl      (x): tensor([[0.5000, 0.5000]]) tensor([[0.2932, 0.2932]])


In [20]:
print('dgl free (y):',dgl_free_dataset.y_normalizer.mean, dgl_free_dataset.y_normalizer.std)
print('dgl      (y):',dgl_dataset.y_normalizer.mean, dgl_dataset.y_normalizer.std)

dgl free (y): tensor([[ 0.0093,  0.0004, -0.0328]], dtype=torch.float64) tensor([[0.2326, 0.1816, 0.1040]], dtype=torch.float64)
dgl      (y): tensor([[ 0.0093,  0.0004, -0.0328]]) tensor([[0.2326, 0.1816, 0.1040]])


In [21]:
print('dgl free (u_p):',dgl_free_dataset.up_normalizer.mean, dgl_free_dataset.up_normalizer.std)
print('dgl      (u_p):',dgl_dataset.up_normalizer.mean, dgl_dataset.up_normalizer.std)

dgl free (u_p): tensor([0.]) tensor([1.0000e-08])
dgl      (u_p): tensor([[0.]]) tensor([[1.0000e-08]])


They all match! all though to be fair, in previous iterations I may not have transformed them correctly. So this may be worth looking at. Lets check the normalizer statistics from our Artemis file:

In [26]:
from train_cavity_artemis import Cavity_2D_dataset_for_GNOT
dataset_args = dict()

# 1. Prepare Data
dataset_args['file']                    = r'C:\Users\Noahc\Documents\USYD\PHD\8 - Github\GNOT\data\steady_cavity_case_b200_maxU100ms_simple_normalized.npy'
dataset_args['percent split (decimal)'] = 0.7
dataset_args['randomizer seed']         = 42
dataset_args['Interpolate (instead of Extrapolate)'] = True
dataset_args['use-normalizer']          = 'unit'
dataset_args['normalize_x']             = 'unit'
dataset_args['cell to pointwise']       = True
dataset_args['add boundaries']          = True
dataset_args['sub_x']                   = 4

dataset = Cavity_2D_dataset_for_GNOT(data_path=dataset_args['file'], 
                                    L=1.0, 
                                    sub_x = dataset_args['sub_x'], 
                                    train=True, 
                                    normalize_y=dataset_args['use-normalizer'], 
                                    y_normalizer=None, 
                                    normalize_x = dataset_args['normalize_x'], 
                                    x_normalizer = None, 
                                    up_normalizer =None, 
                                    vertex = dataset_args['cell to pointwise'], 
                                    boundaries = dataset_args['add boundaries'])
# Process dataset
dataset.assign_data_split_type(inference=dataset_args['Interpolate (instead of Extrapolate)'], 
                                train_ratio=dataset_args['percent split (decimal)'], 
                                seed=dataset_args['randomizer seed'])
dataset.process(theta=False)

Dataset Shape: (200, 65, 65, 3), subsampled by 4
Dataset Split up using torch generator seed: 42
              This can be replicated e.g.
                generator_object = torch.Generator().manual_seed(42)
 
torch.Size([65, 65])
Target features are normalized using unit transformer
tensor([[ 0.0093,  0.0004, -0.0328]], dtype=torch.float64) tensor([[0.2326, 0.1816, 0.1040]], dtype=torch.float64)
Target features are normalized using unit transformer
Input features are normalized using unit transformer


In [27]:
print('dgl free (x):',dgl_free_dataset.x_normalizer.mean, dgl_free_dataset.x_normalizer.std)
print('dgl      (x):',dgl_dataset.x_normalizer.mean, dgl_dataset.x_normalizer.std)
print('artemis  (x):',dataset.x_normalizer.mean, dataset.x_normalizer.std)

dgl free (x): tensor([[0.5000, 0.5000]]) tensor([[0.2932, 0.2932]])
dgl      (x): tensor([[0.5000, 0.5000]]) tensor([[0.2932, 0.2932]])
artemis  (x): tensor([[0.5000, 0.5000]]) tensor([[0.2932, 0.2932]])


In [28]:
print('dgl free (y):',dgl_free_dataset.y_normalizer.mean, dgl_free_dataset.y_normalizer.std)
print('dgl      (y):',dgl_dataset.y_normalizer.mean, dgl_dataset.y_normalizer.std)
print('artemis  (y):',dataset.y_normalizer.mean, dataset.y_normalizer.std)

dgl free (y): tensor([[ 0.0093,  0.0004, -0.0328]], dtype=torch.float64) tensor([[0.2326, 0.1816, 0.1040]], dtype=torch.float64)
dgl      (y): tensor([[ 0.0093,  0.0004, -0.0328]]) tensor([[0.2326, 0.1816, 0.1040]])
artemis  (y): tensor([[ 0.0093,  0.0004, -0.0328]], dtype=torch.float64) tensor([[0.2326, 0.1816, 0.1040]], dtype=torch.float64)


There you got. There is an error the ground truth dataset is not normalized properly

In [30]:
g1_b = dgl.unbatch(g1)
g1_b[0].ndata['x'].shape

# query random point:
print('dgl dataset normalized:', g1_b[0].ndata['x'][100,:])

print('artemis dataset "normalized":',dataset.queries[100,:])

dgl dataset normalized: tensor([ 0.1599, -1.6523])
artemis dataset "normalized": tensor([ 0.1599, -1.6521])


Additionally, it looks like the code itself did not actually transform the dataset! Fixing this might cause the model to train again properly.

**Lets go in and fix this in the artemis code and retry this.**

Now it looks to match! Lets run the artemis training file locally again and hope for the best!

In [34]:
!python train_cavity_artemis.py --theta False

Traceback (most recent call last):
  File "train_cavity_artemis.py", line 404, in <module>
    dataset = CavityDataset(dataset=dataset)
  File "train_cavity_artemis.py", line 234, in __init__
    self.in_queries = self.queries
AttributeError: 'CavityDataset' object has no attribute 'queries'


Dataset Shape: (200, 65, 65, 3), subsampled by 4
Dataset Split up using torch generator seed: 42
              This can be replicated e.g.
                generator_object = torch.Generator().manual_seed(42)
 
torch.Size([65, 65])
Target features are normalized using unit transformer
tensor([[ 0.0093,  0.0004, -0.0328]], dtype=torch.float64) tensor([[0.2326, 0.1816, 0.1040]], dtype=torch.float64)
Target features are normalized using unit transformer
Input features are normalized using unit transformer
